In [1]:
import numpy as np
import cvxpy as cp
from general_method.basis_generator import rand_moment, generate_basis, sel_seq

# Define the Scenario

In [2]:
num_obs = 3
len_seq = 2
out_max = 1
seq_method = "sel_sequences"
remove_last_out = True

In [3]:
sequences = sel_seq(n = num_obs,
                    r = len_seq,
                    out_max = out_max,
                    remove_last_out = remove_last_out)

In [4]:
print(len(sequences))
sequences

9


[((0, 0), (0, 0)),
 ((0, 0), (0, 1)),
 ((0, 0), (0, 2)),
 ((0, 0), (1, 0)),
 ((0, 0), (1, 1)),
 ((0, 0), (1, 2)),
 ((0, 0), (2, 0)),
 ((0, 0), (2, 1)),
 ((0, 0), (2, 2))]

# Sampling Q3 

In [5]:
dim = 2
sel_sequences = [len_seq]

In [6]:
batch_size = 100
X_basis_3d, rank = generate_basis(dim, 
                                  num_obs, 
                                  len_seq, 
                                  out_max, 
                                  batch_size, 
                                  seq_method = seq_method,
                                  sel_sequences = sel_sequences,
                                  remove_last_out=remove_last_out)

rank

35

In [7]:
X = rand_moment(dim, num_obs, len_seq, out_max, seq_method, sel_sequences ,remove_last_out=remove_last_out)
X.shape

(10, 10)

# Generate Q2,NPAk set

In [8]:
dim = 2
level = 1
batch_size = 100

batch_size

100

In [9]:
if level > 1:
    hierarchy_seq = sequences + sel_seq(n = num_obs,
                                    r = len_seq + level -1,
                                    out_max = out_max,
                                    remove_last_out = remove_last_out)
else:
    hierarchy_seq = sequences
    
print(len(hierarchy_seq))
hierarchy_seq

9


[((0, 0), (0, 0)),
 ((0, 0), (0, 1)),
 ((0, 0), (0, 2)),
 ((0, 0), (1, 0)),
 ((0, 0), (1, 1)),
 ((0, 0), (1, 2)),
 ((0, 0), (2, 0)),
 ((0, 0), (2, 1)),
 ((0, 0), (2, 2))]

In [10]:
X_basis_2d, rank = generate_basis(dim = dim, 
                                  num_obs = num_obs, 
                                  len_seq= len_seq + level - 1,
                                  out_max = out_max,
                                  batch_size = batch_size,
                                  seq_method=seq_method,
                                  sel_sequences = [len_seq, len_seq + level - 1],
                                  remove_last_out=remove_last_out)

print(X_basis_2d[0].shape)
rank

(10, 10)


35

In [11]:
#X_basis_2d = []

for i in range(500):
    X_basis_2d.append(rand_moment(dim, num_obs, len_seq, out_max, seq_method, [len_seq, len_seq + level - 1], remove_last_out=remove_last_out))

# Testing via SDP

In [12]:
eta = cp.Variable((1, 1))
alpha = cp.Variable((len(X_basis_2d), 1))

constraints = []

for i in range(1,len(X)):
    constraints += [
        eta*X[i,i] + (1-eta)*X_basis_2d[0][i,i] == sum([alpha[i]*X_basis_2d[i] for i in range(len(X_basis_2d))])[i,i] 
    ]

prob = cp.Problem(cp.Maximize(eta),
                  constraints)

prob.solve(verbose=True)

-----------------------------------------------------------------
           OSQP v0.5.0  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2018
-----------------------------------------------------------------
problem:  variables n = 601, constraints m = 18
          nnz(P) + nnz(A) = 5418
settings: linear system solver = qdldl,
          eps_abs = 1.0e-05, eps_rel = 1.0e-05,
          eps_prim_inf = 1.0e-04, eps_dual_inf = 1.0e-04,
          rho = 1.00e-01 (adaptive),
          sigma = 1.00e-06, alpha = 1.60, max_iter = 10000
          check_termination: on (interval 25),
          scaling: on, scaled_termination: off
          warm start: on, polish: on

iter   objective    pri res    dua res    rho        time
   1  -2.4672e+06   8.04e-01   5.12e+02   1.00e-01   6.27e-04s
  25  -1.0000e+20   2.48e-04   9.53e-01   1.00e-01   9.89e-04s

status:               dual infeasible
number of iterations: 2

inf